In [1]:
import torch
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader
import timm  # For Xception model

# Device configuration (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 1. Define model architectures
# ResNet18 with 2 output classes
model1 = models.resnet18(pretrained=False)
model1.fc = torch.nn.Linear(model1.fc.in_features, 2)  # Modify FC layer for 2 classes

# DenseNet201 with 2 output classes
model4 = models.densenet201(pretrained=False)
model4.classifier = torch.nn.Linear(model4.classifier.in_features, 2)  # Modify classifier for 2 classes

# Xception model with 2 output classes (from TIMM)
model5 = timm.create_model('xception', pretrained=False)
model5.fc = torch.nn.Linear(model5.fc.in_features, 2)  # Modify FC layer for 2 classes

# 2. Load trained weights (replace with actual file paths)
model1.load_state_dict(torch.load('resnet_best_model_combined.pth', map_location=device))
model4.load_state_dict(torch.load('Densenet__weight.pth', map_location=device))
model5.load_state_dict(torch.load('xception_best_model_combined.pth', map_location=device), strict=False)

# 3. Move models to device (CPU or GPU)
model1.to(device)
model4.to(device)
model5.to(device)

# 4. Set models to evaluation mode
model1.eval()
model4.eval()
model5.eval()

# Define transformations for 224x224 version of the test dataset
transform_224 = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize for ResNet18, DenseNet201, Xception
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Replace with the actual path to your test dataset
test_dataset_224 = datasets.ImageFolder(root="./test3", transform=transform_224)

# DataLoader for test dataset
test_dl_224 = DataLoader(test_dataset_224, batch_size=32, shuffle=False)

# 5. Initialize counters for correct predictions for each model
correct_model1 = correct_model4 = correct_model5 = correct_ensemble = 0
total = 0

# Evaluate accuracy of each model and the ensemble
with torch.no_grad():
    for xb_224, yb in test_dl_224:
        xb_224, yb = xb_224.to(device), yb.to(device)
        
        # Get predictions from each model
        outputs1 = model1(xb_224)
        outputs4 = model4(xb_224)
        outputs5 = model5(xb_224)
        
        # Calculate individual predictions and update correct counts
        preds1 = torch.argmax(outputs1, dim=1)
        preds4 = torch.argmax(outputs4, dim=1)
        preds5 = torch.argmax(outputs5, dim=1)
        
        correct_model1 += (preds1 == yb).sum().item()
        correct_model4 += (preds4 == yb).sum().item()
        correct_model5 += (preds5 == yb).sum().item()
        
        # Averaging the predictions from 3 models for ensemble
        avg_outputs = (outputs1 + outputs4 + outputs5) / 3
        preds_ensemble = torch.argmax(avg_outputs, dim=1)
        
        correct_ensemble += (preds_ensemble == yb).sum().item()
        total += yb.size(0)

# Calculate and print accuracies
accuracy_model1 = correct_model1 / total * 100
accuracy_model4 = correct_model4 / total * 100
accuracy_model5 = correct_model5 / total * 100
ensemble_accuracy = correct_ensemble / total * 100

print(f'Model 1 (ResNet18) Accuracy: {accuracy_model1:.2f}%')
print(f'Model 4 (DenseNet201) Accuracy: {accuracy_model4:.2f}%')
print(f'Model 5 (Xception) Accuracy: {accuracy_model5:.2f}%')
print(f'Ensemble Test Accuracy: {ensemble_accuracy:.2f}%')


C:\Users\SK\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\SK\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
C:\Users\SK\anaconda3\Lib\site-packages\timm\models\_factory.py:117: UserWarning: Mapping deprecated model name xception to current legacy_xception.
  model = create_fn(
C:\Users\SK\AppData\Local\Temp\ipykernel_1844\3215966129.py:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See http

Model 1 (ResNet18) Accuracy: 98.50%
Model 4 (DenseNet201) Accuracy: 98.17%
Model 5 (Xception) Accuracy: 99.17%
Ensemble Test Accuracy: 99.00%
